In [2]:
import wget
from sentence_transformers import SentenceTransformer, util
import torch
import pandas as pd
import numpy as np
import seaborn as sns
import pickle
import time
from transformers import AutoTokenizer, AutoModel
from transformers import BertTokenizer, BertModel

C:\Users\User\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\User\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\User\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\User\AppData\Local\Continuum

In [3]:
def tokenize(model, tokenizer, sentence):
    
    # Tokens comes from a process that splits the input into sub-entities with interesting linguistic properties.
    tokens = tokenizer.tokenize(sentence.lower())

    # This is not sufficient for the model, as it requires integers as input, 
    # not a problem, let's convert tokens to ids.
    tokens_ids = tokenizer.convert_tokens_to_ids(tokens)
    #print(tokens_ids)

    # Add the required special tokens
    tokens_ids = tokenizer.build_inputs_with_special_tokens(tokens_ids)
    #print(tokens_ids)
    
    # We need to convert to a Deep Learning framework specific format, let's use PyTorch for now.
    tokens_pt = torch.tensor([tokens_ids])

    # The length of the tokens can not be more than 512
    if len(tokens_pt[0]) > 512:
        tokens_pt = torch.tensor([tokens_pt[0][0:512].tolist()])
        
    # Now we're ready to go through BERT with out input
    outputs, pooled = model(tokens_pt)
    return pooled

## Loading in the models

In [4]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

In [11]:
scibert_tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_uncased")
scibert_model = AutoModel.from_pretrained("allenai/scibert_scivocab_uncased")

In [12]:
covid_tokenizer = AutoTokenizer.from_pretrained("gsarti/covidbert-nli")
covid_model = AutoModel.from_pretrained("gsarti/covidbert-nli")

In [8]:
tok = bert_tokenizer.tokenize("[SEP]")
bert_tokenizer.convert_tokens_to_ids(tok)

[102]

## Loading in the data

In [4]:
path = "C:\\Users\\User\\OneDrive - NTNU\\NTNU\\Prosjekt oppgave NLP\\dataset\\CORD-19\\"
docs = pd.read_csv(path + "crod_19_only_rel.csv")
docs.shape
docs.head()

C:\Users\User\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,...,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id,body_text
0,3,2b73a28n,348055649b6b8cf2b9a376498df9bf41f7123605,PMC,Role of endothelin-1 in lung disease,10.1186/rr44,PMC59574,11686871.0,no-cc,Endothelin-1 (ET-1) is a 21 amino acid peptide...,...,"Fagan, Karen A; McMurtry, Ivan F; Rodman, David M",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/348055649b6b8cf2b9a37...,document_parses/pmc_json/PMC59574.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN,from xenopus laevis [16] . eta receptors in no...
1,5,zjufx4fo,b2897e1277f56641193a6db73825f707eed3e4c9,PMC,Sequence requirements for RNA strand transfer ...,10.1093/emboj/20.24.7220,PMC125340,11742998.0,green-oa,Nidovirus subgenomic mRNAs contain a leader se...,...,"Pasternak, Alexander O.; van den Born, Erwin; ...",The EMBO Journal,NaN,NaN,NaN,document_parses/pdf_json/b2897e1277f56641193a6...,document_parses/pmc_json/PMC125340.xml.json,http://europepmc.org/articles/pmc125340?pdf=re...,NaN,the genetic information of rna viruses is orga...
2,7,8zchiykl,5806726a24dc91de3954001effbdffd7a82d54e2,PMC,The 21st International Symposium on Intensive ...,10.1186/cc1013,PMC137274,11353930.0,no-cc,The 21st International Symposium on Intensive ...,...,"Ball, Jonathan; Venn, Richard",Crit Care,NaN,NaN,NaN,document_parses/pdf_json/5806726a24dc91de39540...,document_parses/pmc_json/PMC137274.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN,this year's symposium was dominated by the res...
3,10,5tkvsudh,9d4e3e8eb092d5ed282d0aa4aadcaa8b7165b5e9,PMC,Conservation of polyamine regulation by transl...,10.1093/emboj/19.8.1907,PMC302018,10775274.0,no-cc,Regulation of ornithine decarboxylase in verte...,...,"Ivanov, Ivaylo P.; Matsufuji, Senya; Murakami,...",EMBO J,NaN,NaN,NaN,document_parses/pdf_json/9d4e3e8eb092d5ed282d0...,document_parses/pmc_json/PMC302018.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN,the ef®ciency of +1 ribosomal frameshifting at...
4,12,tvxpckxo,d09b79026117ec9faebba46a8d13aa9b23ec751e,PMC,A Method to Identify p62's UBA Domain Interact...,10.1251/bpo66,PMC302190,14702098.0,no-cc,The UBA domain is a conserved sequence motif a...,...,"Pridgeon, Julia W.; Geetha, Thangiah; Wooten, ...",Biol Proced Online,NaN,NaN,NaN,document_parses/pdf_json/d09b79026117ec9faebba...,document_parses/pmc_json/PMC302190.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN,p62 is a novel cellular protein which was init...


In [ ]:
docs.head(2)

## Bert embeddings

### Title

In [ ]:
path = "C:/Users/User/Documents/NTNU/NLP/CORD-19/Embeddings/BERT/786/title/"
for idx, row in docs.iterrows():
    BERT_embeddings_title = {}
    sha = row["sha"]
    cord_uid = row["cord_uid"]
    BERT_embeddings_title[row["sha"]] = tokenize(bert_model, bert_tokenizer, row["title"])
    if (idx % 10000) == 0:
        print(idx)
    file = path + cord_uid + ".txt"
    with open(file, "wb") as fp:   #Pickling
        pickle.dump(BERT_embeddings_title, fp)

In [ ]:
path = "C:\\Users\\User\\OneDrive - NTNU\\NTNU\\Prosjekt oppgave NLP\\dataset\\CORD-19\\embeddings\\"

with open(path + "BERT_embedding_title.txt", "wb") as fp:   #Pickling
    pickle.dump(BERT_embeddings_title, fp)
    

### Abstract

In [ ]:
BERT_embeddings_abstract = {}
for idx, row in docs.iterrows():
    BERT_embeddings_abstract[row["sha"]] = tokenize(bert_model, bert_tokenizer, row["abstract"])
    if (idx % 10000) == 0:
        print(idx)

In [ ]:
path = "C:\\Users\\User\\OneDrive - NTNU\\NTNU\\Prosjekt oppgave NLP\\dataset\\CORD-19\\embeddings\\"

with open(path + "BERT_embedding_abstract.txt", "wb") as fp:   #Pickling
    pickle.dump(BERT_embeddings_abstract, fp)

## SciBert embeddings

### Title

In [ ]:
SciBERT_embeddings_title = {}
for idx, row in docs.iterrows():
    SciBERT_embeddings_title[row["sha"]] = tokenize(scibert_model, scibert_tokenizer, row["title"])
    if (idx % 10000) == 0:
        print(idx)

In [ ]:
path = "C:\\Users\\User\\OneDrive - NTNU\\NTNU\\Prosjekt oppgave NLP\\dataset\\CORD-19\\embeddings\\"

with open(path + "SciBERT_embedding_title.txt", "wb") as fp:   #Pickling
    pickle.dump(SciBERT_embeddings_title, fp)

### Abstract

In [ ]:
SciBERT_embeddings_abstract = {}
for idx, row in docs.iterrows():
    SciBERT_embeddings_abstract[row["sha"]] = tokenize(scibert_model, scibert_tokenizer, row["abstract"])
    if (idx % 10000) == 0:
        print(idx)

In [ ]:
path = "C:\\Users\\User\\OneDrive - NTNU\\NTNU\\Prosjekt oppgave NLP\\dataset\\CORD-19\\embeddings\\"

with open(path + "SciBERT_embedding_abstract.txt", "wb") as fp:   #Pickling
    pickle.dump(SciBERT_embeddings_abstract, fp)

## CordBert embeddings

### Title

In [ ]:
CordBERT_embeddings_title = {}
for idx, row in docs.iterrows():
    CordBERT_embeddings_title[row["sha"]] = tokenize(covid_model, covid_tokenizer, row["title"])
    if (idx % 10000) == 0:
        print(idx)

In [ ]:
path = "C:\\Users\\User\\OneDrive - NTNU\\NTNU\\Prosjekt oppgave NLP\\dataset\\CORD-19\\embeddings\\"

with open(path + "CordBERT_embedding_title.txt", "wb") as fp:   #Pickling
    pickle.dump(CordBERT_embeddings_title, fp)

### Abstract

In [ ]:
CordBERT_embeddings_abstract = {}
for idx, row in docs.iterrows():
    CordBERT_embeddings_abstract[row["sha"]] = tokenize(covid_model, covid_tokenizer, row["abstract"])
    if (idx % 10000) == 0:
        print(idx)

In [ ]:
path = "C:\\Users\\User\\OneDrive - NTNU\\NTNU\\Prosjekt oppgave NLP\\dataset\\CORD-19\\embeddings\\"

with open(path + "CordBERT_embedding_abstract.txt", "wb") as fp:   #Pickling
    pickle.dump(CordBERT_embeddings_abstract, fp)

## Embedding dim=786

In [5]:
def isNaN(num):
    return num != num

In [69]:
path = "C:/Users/User/Documents/NTNU/NLP/CORD-19/Embeddings/786/"
n_models = 3
n_fields = 2
dim = 786

start = time.time()
m = start

for idx, row in docs.iterrows():
    embedding = {}
    if (idx>21999):
        sha = row["sha"]
        cord_uid = row["cord_uid"]
        embedding["sha"] = sha
        embedding["cord_uid"] = cord_uid
        embedding["dim"] = dim
        embedding["models"] = {}

        models = [bert_model, scibert_model, covid_model]
        model_name = ['bert-base-uncased', "allenai/scibert_scivocab_uncased", "gsarti/covidbert-nli"]
        tokenizers = [bert_tokenizer ,scibert_tokenizer, covid_tokenizer]
        fields = ["title", "abstract"]

        for i in range(n_models):
            embedding["models"][model_name[i]] = {}
            for j in range(n_fields):
                field = row[fields[j]]

                res = isinstance(fields[j], str) 
                if (not res) or isNaN(field):
                    field = row["title"]
              
                
                embedd = tokenize(models[i], tokenizers[i], field)
                embedding["models"][model_name[i]][fields[j]] = embedd
                torch.cuda.empty_cache()

        ## Timing the loops
        if (idx % 1000) == 0:
            print(idx)
            print("Time (1000 iterations):", round(time.time() - m,1))
            m = time.time()

        file = path + cord_uid + ".txt"
        with open(file, "wb") as fp:   #Pickling
            pickle.dump(embedding, fp)

22000
Time (1000 iterations): 2.9
23000
Time (1000 iterations): 2901.4
24000
Time (1000 iterations): 3052.3
25000
Time (1000 iterations): 3092.0
26000
Time (1000 iterations): 3273.8
27000
Time (1000 iterations): 3098.5
28000
Time (1000 iterations): 3359.0
29000
Time (1000 iterations): 3308.5
30000
Time (1000 iterations): 2962.8
31000
Time (1000 iterations): 3120.6
32000
Time (1000 iterations): 3144.9
33000
Time (1000 iterations): 2534.2
34000
Time (1000 iterations): 2870.9
35000
Time (1000 iterations): 2798.4
36000
Time (1000 iterations): 2563.8
37000
Time (1000 iterations): 2757.5


In [15]:
path = "C:/Users/User/Documents/NTNU/NLP/CORD-19/Embeddings/786/"

s = time.time()
for idx, row in docs.iterrows():
    cord_uid = row["cord_uid"]
    file = path + cord_uid + ".txt"
    with open(file, "rb") as fp:   # Unpickling
        embedding = pickle.load(fp)
        print(embedding)
    if (idx >10):
        break
print(time.time()-s)

{'sha': '348055649b6b8cf2b9a376498df9bf41f7123605', 'cord_uid': '2b73a28n', 'dim': 786, 'models': {'bert-base-uncased': {'title': tensor([[-0.8821, -0.4513, -0.7262,  0.5963,  0.4377, -0.2772,  0.5105,  0.4937,
         -0.6845, -1.0000, -0.4603,  0.8741,  0.9803,  0.2004,  0.8190, -0.6054,
         -0.2227, -0.5334,  0.4174,  0.2318,  0.5804,  0.9999, -0.1095,  0.3830,
          0.5117,  0.9426, -0.6965,  0.7879,  0.9346,  0.7325, -0.5224,  0.5704,
         -0.9822, -0.3218, -0.7710, -0.9892,  0.5202, -0.6966, -0.1225, -0.2924,
         -0.6803,  0.5299,  1.0000, -0.1353,  0.5102, -0.4058, -1.0000,  0.3844,
         -0.8318,  0.8088,  0.6565,  0.7812,  0.3226,  0.5875,  0.6141, -0.6616,
          0.0863,  0.2363, -0.4666, -0.6441, -0.6983,  0.6352, -0.6398, -0.8916,
          0.7789,  0.5303, -0.3908, -0.5642, -0.2168,  0.1401,  0.6407,  0.3567,
         -0.1510, -0.7890,  0.5563,  0.4103, -0.6734,  1.0000, -0.5632, -0.9739,
          0.4509,  0.6616,  0.6511, -0.1534,  0.1932, -1.000

{'sha': 'd09b79026117ec9faebba46a8d13aa9b23ec751e', 'cord_uid': 'tvxpckxo', 'dim': 786, 'models': {'bert-base-uncased': {'title': tensor([[-9.0962e-01, -2.4211e-01, -5.7992e-01,  6.6346e-01,  4.2732e-01,
         -2.0304e-01,  7.2649e-01,  2.4327e-01, -3.7300e-01, -9.9997e-01,
         -2.9289e-01,  8.4341e-01,  9.8308e-01, -2.8110e-02,  9.0299e-01,
         -6.1760e-01, -6.4273e-02, -4.9666e-01,  2.3233e-01, -3.0744e-01,
          7.2121e-01,  9.9990e-01,  1.1629e-01,  2.7226e-01,  3.9849e-01,
          9.3556e-01, -6.8103e-01,  9.1849e-01,  9.4838e-01,  7.8392e-01,
         -6.5405e-01,  3.3325e-01, -9.8703e-01, -1.4627e-01, -6.7630e-01,
         -9.9254e-01,  4.1189e-01, -7.4710e-01,  1.0250e-01, -5.9331e-02,
         -8.5439e-01,  4.4840e-01,  9.9991e-01, -3.0105e-01,  4.6464e-01,
         -2.3629e-01, -1.0000e+00,  2.2417e-01, -8.6332e-01,  5.8278e-01,
          4.8692e-01,  6.4451e-01,  1.8024e-01,  5.4596e-01,  4.0938e-01,
         -3.1846e-01, -9.3745e-02,  1.1342e-01, -2.1172e

         -9.3496e-01, -2.6605e-01,  2.1338e-01]], requires_grad=True)}}}
{'sha': '867e1b0f6ca8757f2a32a625d99b23888ab40d49', 'cord_uid': 'bbvxu8op', 'dim': 786, 'models': {'bert-base-uncased': {'title': tensor([[-8.6451e-01, -3.1468e-01, -9.6733e-01,  5.4751e-01,  8.3157e-01,
         -4.2815e-01, -3.0098e-02,  1.8505e-01, -8.1319e-01, -9.9987e-01,
         -5.2616e-01,  9.7545e-01,  9.5132e-01,  4.7262e-01,  6.3935e-01,
         -5.6997e-01, -4.5607e-01, -3.9607e-01,  2.9592e-01,  6.7091e-01,
          6.8230e-01,  1.0000e+00, -5.1778e-01,  3.5526e-01,  3.2400e-01,
          9.8891e-01, -7.5388e-01,  7.1988e-01,  8.9163e-01,  7.6728e-01,
         -2.9698e-01,  3.8250e-01, -9.8081e-01, -9.7437e-02, -9.7989e-01,
         -9.8015e-01,  5.2855e-01, -5.5545e-01,  1.9613e-02,  5.3681e-02,
         -7.3549e-01,  5.4938e-01,  9.9996e-01, -5.5958e-02,  7.6454e-01,
         -2.5673e-01, -1.0000e+00,  2.9447e-01, -6.8401e-01,  9.3561e-01,
          9.0201e-01,  9.6522e-01,  2.8856e-01,  5.4969e-

{'sha': 'ae19236936b05793ceac08ed5d58269c25486320', 'cord_uid': 'zwbc7nnn', 'dim': 786, 'models': {'bert-base-uncased': {'title': tensor([[-8.9576e-01, -4.3610e-01, -9.3057e-01,  7.4483e-01,  7.3880e-01,
         -2.1552e-01,  5.6254e-01,  2.6832e-01, -8.0379e-01, -9.9999e-01,
         -6.1890e-01,  9.3047e-01,  9.6340e-01,  4.8147e-01,  7.6390e-01,
         -7.2162e-01,  5.1347e-03, -5.6238e-01,  2.5695e-01,  2.4411e-01,
          5.9768e-01,  1.0000e+00, -3.0874e-01,  3.9732e-01,  3.9800e-01,
          9.8188e-01, -7.1866e-01,  8.1730e-01,  9.3003e-01,  7.5349e-01,
         -4.3267e-01,  3.7293e-01, -9.7852e-01, -1.5450e-01, -9.3489e-01,
         -9.8912e-01,  4.8337e-01, -6.8362e-01,  6.6629e-02, -9.2893e-02,
         -7.5198e-01,  4.9161e-01,  9.9999e-01,  1.2312e-02,  6.3091e-01,
         -2.7091e-01, -1.0000e+00,  2.5499e-01, -8.1781e-01,  9.3566e-01,
          8.7415e-01,  9.4338e-01,  2.1927e-01,  4.7723e-01,  4.9842e-01,
         -6.2799e-01, -1.4479e-01,  1.3415e-01, -3.2365e

In [18]:
path = "C:/Users/User/Documents/NTNU/NLP/CORD-19/Embeddings/786/"

s = time.time()
for idx, row in docs.iterrows():
    cord_uid = row["cord_uid"]
    file = path + cord_uid + ".txt"
    with open(file, "rb") as fp:   # Unpickling
        embedding = pickle.load(fp)
        print(embedding["models"]["bert-base-uncased"]["abstract"][0])
    if (idx >10):
        break
print(time.time()-s)

tensor([-7.2691e-01, -2.3869e-01, -8.6512e-01,  2.0167e-01,  6.9648e-01,
         8.9377e-02, -6.6326e-01,  4.8240e-02, -8.8421e-01, -9.9984e-01,
        -6.2786e-01,  9.2084e-01,  8.9756e-01, -1.6120e-01,  3.8130e-01,
        -3.1916e-01,  3.6126e-01,  8.2679e-03,  8.8782e-02,  9.3996e-01,
         1.6620e-02,  9.9996e-01, -3.2176e-01,  1.8783e-01,  1.3297e-02,
         9.6083e-01, -5.8463e-01,  5.0327e-01,  7.3296e-01,  3.4668e-01,
         3.3784e-01,  9.8785e-02, -9.1816e-01,  2.8031e-01, -9.1155e-01,
        -9.1367e-01,  2.9899e-01, -3.0932e-01,  4.1385e-01,  5.0384e-03,
        -3.7188e-01,  1.5383e-01,  9.9998e-01, -6.1542e-01,  4.3890e-01,
         7.4744e-02, -9.9930e-01, -2.8538e-02, -4.3428e-01,  9.3243e-01,
         7.4099e-01,  9.5736e-01, -1.0811e-02,  2.1400e-01,  2.5949e-01,
        -6.3800e-01, -4.2442e-01, -1.1899e-01, -6.0226e-02, -2.6048e-01,
        -2.3630e-01, -8.0318e-05, -8.0428e-01, -7.9991e-01,  9.3150e-01,
         7.1275e-01,  9.5864e-02, -4.2324e-02,  3.5

2.5659639835357666


In [17]:
path = "C:/Users/User/Documents/NTNU/NLP/CORD-19/Embeddings/786/"
n_models = 3
n_fields = 1
dim = 786

start = time.time()
m = start

for idx, row in docs.iterrows():
    embedding = {}
    if (idx<22000):
        cord_uid = row["cord_uid"]
        embedding = {}
        file = path + cord_uid + ".txt"
        with open(file, "rb") as fp:   # Unpickling
            embedding = pickle.load(fp)
       
    
        models = [bert_model, scibert_model, covid_model]
        model_name = ['bert-base-uncased', "allenai/scibert_scivocab_uncased", "gsarti/covidbert-nli"]
        tokenizers = [bert_tokenizer ,scibert_tokenizer, covid_tokenizer]
        fields = ["abstract"]

        for i in range(n_models):
            for j in range(n_fields):
                field = row[fields[j]]

                res = isinstance(fields[j], str) 
                if (not res) or isNaN(field):
                    field = row["title"]
              
                
                embedd = tokenize(models[i], tokenizers[i], field)
                embedding["models"][model_name[i]][fields[j]] = embedd
                torch.cuda.empty_cache()

        ## Timing the loops
        if (idx % 1000) == 0:
            print(idx)
            print("Time (1000 iterations):", round(time.time() - m,1))
            m = time.time()

        with open(file, "wb") as fp:   #Pickling
            pickle.dump(embedding, fp)

0
Time (1000 iterations): 0.8
1000
Time (1000 iterations): 2745.9


KeyboardInterrupt: 

In [ ]:
"""
{
sha: sha,
cord_uid: cord_uid,
dim: dim
models: {model_name1: {title: [...],
                       abstract: [...]},
         model_name2: {title: [...],
                       abstract: [...]},
        ...                   
}
}


"""

In [ ]:
torch.cuda.empty_cache()

## Embedding topics

In [21]:
# loading topics data

path = 'C:/Users/User/OneDrive - NTNU/NTNU/Prosjekt oppgave NLP/dataset/CORD-19/'
file = 'topics.csv'

topics = pd.read_csv(path + file)
topics.head()

,query_id,query,question,narrative
0,1,coronavirus origin,what is the origin of COVID-19,seeking range of information about the SARS-Co...
1,2,coronavirus response to weather changes,how does the coronavirus respond to changes in...,seeking range of information about the SARS-Co...
2,3,coronavirus immunity,will SARS-CoV2 infected people develop immunit...,seeking studies of immunity developed due to i...
3,4,how do people die from the coronavirus,what causes death from Covid-19?,Studies looking at mechanisms of death from Co...
4,5,animal models of COVID-19,what drugs have been active against SARS-CoV o...,Papers that describe the results of testing d...


In [25]:
path = "C:/Users/User/Documents/NTNU/NLP/CORD-19/topic_embeddings/"
n_models = 3
n_fields = 3
dim = 786

start = time.time()
m = start

for idx, row in topics.iterrows():
    embedding = {}
    embedding["topic_id"] = row["query_id"]
    embedding["dim"] = dim
    embedding["models"] = {}

    models = [bert_model, scibert_model, covid_model]
    model_name = ['bert-base-uncased', "allenai/scibert_scivocab_uncased", "gsarti/covidbert-nli"]
    tokenizers = [bert_tokenizer ,scibert_tokenizer, covid_tokenizer]
    fields = ["query", "question", "narrative"]

    for i in range(n_models):
        embedding["models"][model_name[i]] = {}
        for j in range(n_fields):
            field = row[fields[j]]



            embedd = tokenize(models[i], tokenizers[i], field)
            embedding["models"][model_name[i]][fields[j]] = embedd

    ## Timing the loops
    if (idx % 1000) == 0:
        print(idx)
        print("Time (1000 iterations):", round(time.time() - m,1))
        m = time.time()

    file = path + str(row["query_id"]) + ".txt"
    with open(file, "wb") as fp:   #Pickling
        pickle.dump(embedding, fp)

0
Time (1000 iterations): 0.8
